In [2]:
import pysam
import pandas as pd
import re

In [15]:
def read_gold(fn, d):
    f_gold = pysam.AlignmentFile(fn, 'r')
    for r in f_gold:
        if r.is_read1:
            d[r.query_name + '_1'] = [r.reference_name, r.reference_start]
        elif r.is_read2:
            d[r.query_name + '_2'] = [r.reference_name, r.reference_start]
    f_gold.close()
    return d
    
def read_query(fn, d):
    f = pysam.AlignmentFile(fn, 'r')
    for r in f:
        if r.is_supplementary or r.is_secondary:
            continue
        declip_pos = r.reference_start
        if r.cigarstring.count('S') > 0:
            re_cigar = re.compile('[SMID+]')
            if re.findall(re_cigar, r.cigarstring)[0] == 'S':
                declip_pos = r.reference_start - int(re.split(re_cigar, r.cigarstring)[0])
            
        info = [r.flag, r.reference_name, r.reference_start, r.mapping_quality,
                 r.get_tag("AS") if r.has_tag("AS") else None, r.get_tag("NM") if r.has_tag("NM") else None, r.cigarstring, declip_pos]
        if r.is_read1:
            d[r.query_name + '_1'].extend(info)
        elif r.is_read2:
            d[r.query_name + '_2'].extend(info)
    f.close()
    df = pd.DataFrame.from_dict(d, orient='index', columns=['gold_rname', 'gold_pos', 'flag', 'rname', 'pos', 'mapq', 'score', 'hdist', 'cigar', 'pos_noclip'])
    return df

In [61]:
d = read_gold('chr21-per.bam', {})
df = read_query('bwa-chm13_to_grch38-final.bam', d)
# df = pd.DataFrame.from_dict(d, orient='index', columns=['gold_rname', 'gold_pos', 'flag', 'rname', 'pos', 'mapq', 'score', 'hdist'])
# df = pd.DataFrame.from_dict(d, orient='index', columns=['gold_rname', 'gold_pos', 'lev_flag', 'lev_rname', 'lev_pos', 'lev_mapq', 'grc_flag', 'grc_rname', 'grc_pos', 'grc_mapq'])
df.to_csv('NA12878-bwa-lev.tsv', sep='\t')


In [62]:
d = read_gold('chr21-per.bam', {})
df = read_query('bwa-chm13_to_grch38-final-ns.bam', d)

# df = pd.DataFrame.from_dict(d, orient='index', columns=['gold_rname', 'gold_pos', 'flag', 'rname', 'pos', 'mapq', 'score', 'hdist'])
# df = pd.DataFrame.from_dict(d, orient='index', columns=['gold_rname', 'gold_pos', 'lev_flag', 'lev_rname', 'lev_pos', 'lev_mapq', 'grc_flag', 'grc_rname', 'grc_pos', 'grc_mapq'])
df.to_csv('NA12878-bwa-lev-ns.tsv', sep='\t')


In [63]:
# d = read_gold('chr21-per.bam', {})
# df = read_query('bwa-chm13_to_grch38-final-lm.bam', d)

# # df = pd.DataFrame.from_dict(d, orient='index', columns=['gold_rname', 'gold_pos', 'flag', 'rname', 'pos', 'mapq', 'score', 'hdist'])
# # df = pd.DataFrame.from_dict(d, orient='index', columns=['gold_rname', 'gold_pos', 'lev_flag', 'lev_rname', 'lev_pos', 'lev_mapq', 'grc_flag', 'grc_rname', 'grc_pos', 'grc_mapq'])
# df.to_csv('NA12878-bwa-lev-lm.tsv', sep='\t')


In [64]:
d = read_gold('chr21-per.bam', {})
df = read_query('bwa-grch38.bam', d)

# df = pd.DataFrame.from_dict(d, orient='index', columns=['gold_rname', 'gold_pos', 'flag', 'rname', 'pos', 'mapq', 'score', 'hdist'])
df.to_csv('NA12878-bwa-grc.tsv', sep='\t')

In [32]:
r.cigarstring

'100M'

In [49]:
re_cigar = re.compile('[SMID+]')
print(re.findall(re_cigar, '5S80M1D15M'))
print(re.split(re_cigar, '5S80M1D15M'))

print(re.findall(re_cigar, '80M1D15M5S'))
print(re.split(re_cigar, '80M1D15M5S'))

['S', 'M', 'D', 'M']
['5', '80', '1', '15', '']
['M', 'D', 'M', 'S']
['80', '1', '15', '5', '']


In [7]:
d = read_gold('10m/chr21-per.bam', {})
df = read_query('10m/bwa-chm13_to_grch38-final.bam', d)
df.to_csv('10m/NA12878-bwa-lev.tsv', sep='\t')


AssertionError: 10 columns passed, passed data had 18 columns

In [16]:
d = read_gold('10m/chr21-per.bam', {})
df = read_query('10m/bwa-grch38.bam', d)
df.to_csv('10m/NA12878-bwa-grc.tsv', sep='\t')


In [14]:
for i, (k, v) in enumerate(d.items()):
    if len(v) != 10:
        print((k,v))

('simulated.728507_2', ['chr21', 26311370, 147, 'chr21', 26311459, 0, 67, 0, '33S67M', 26311426, 2179, 'chr18', 21268578, 0, 32, 1, '58H37M5H', 21268578])
('simulated.841431_2', ['chr21', 32470824, 147, 'chr21', 32470824, 14, 74, 1, '76M24S', 32470824, 2179, 'chr1', 197042671, 0, 36, 1, '41M59H', 197042671])
('simulated.1128168_2', ['chr21', 32470877, 129, 'chr16', 25092886, 0, 66, 1, '71M29S', 25092886, 2193, 'chr5', 178300184, 0, 38, 5, '23M4D30M47H', 178300184])
('simulated.1437014_1', ['chr21', 35419943, 99, 'chr21', 35419968, 60, 75, 0, '25S75M', 35419943, 2163, 'chr11', 122989239, 0, 41, 1, '58H42M', 122989239])
('simulated.1560589_2', ['chr21', 32470874, 177, 'chr16', 25092889, 2, 63, 1, '68M32S', 25092889, 2209, 'chr7', 6808824, 0, 37, 4, '56M44H', 6808824])
('simulated.1843451_2', ['chr21', 19175455, 145, 'chr16', 73606094, 0, 62, 0, '38S62M', 73606056, 2177, 'chr10', 14626633, 0, 42, 0, '58H42M', 14626633])
('simulated.1943932_2', ['chr21', 19232527, 147, 'chr21', 19232527, 6